In [1]:
import pandas as pd
import numpy as np
import datetime as dt

from sklearn.preprocessing import LabelEncoder 
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler

In [2]:
#Importamos los set de datos.
eventos = pd.read_csv("../events_up_to_01062018.csv", low_memory=False)
labels = pd.read_csv("../trocafone_kaggle_test.csv")

In [3]:
list(eventos)

['timestamp',
 'event',
 'person',
 'url',
 'sku',
 'model',
 'condition',
 'storage',
 'color',
 'skus',
 'search_term',
 'staticpage',
 'campaign_source',
 'search_engine',
 'channel',
 'new_vs_returning',
 'city',
 'region',
 'country',
 'device_type',
 'screen_resolution',
 'operating_system_version',
 'browser_version']

In [17]:
#Creamos un dataframe de features
features = labels

In [18]:
features.shape

(19415, 1)

In [19]:
#Feature: Es un usuario que retorno alguna vez?

df = eventos.groupby(['person'])['new_vs_returning'].agg('count').reset_index()
df['retornoAlSitio'] = df["new_vs_returning"].apply(lambda x: True if x>0 else False)
del df['new_vs_returning']
features = pd.merge(features, df, on='person', how= 'left')


In [20]:
#Feature: Cantidad de tipo de estado de dispositivo que busco.

eventos_visualizacion = eventos.loc[eventos.event == 'viewed product']
df = eventos_visualizacion.groupby('person')['condition'].value_counts().unstack()
df = df.rename(columns={'Bom': 'cantidadVisualizacionesBuenEstado', 
                 'Bom - Sem Touch ID': 'cantidadVisualizacionesBuenEstadoSinTouch', 
                 'Novo': 'cantidadVisualizacionNuevo',
                 'Muito Bom': 'cantidadVisualizacionesMuyBuenEstado',
                 'Excelente': 'cantidadVisualizacionesExcelente'})

df = df.fillna(0)
scaler = MinMaxScaler()
for column in df:
    df[[column]] = scaler.fit_transform(df[[column]])

features = pd.merge(features, df, on='person', how= 'left')

In [21]:
#Feature: Cantidad de eventos generados totales

df = eventos.groupby(['person'])['event'].agg('count').to_frame('cantidad_eventos_generados')
scaler = MinMaxScaler()
for column in df:
    df[[column]] = scaler.fit_transform(df[[column]])

features = pd.merge(features, df, on='person', how= 'left')



In [22]:
#Features: Cantidad por tipo de evento generado
df = eventos.groupby('person')['event'].value_counts().unstack()
df = df.fillna(0)
df = df.rename(columns={'ad campaign hit': 'cantidadIngresosPorCampania', 
                 'brand listing': 'cantidadDeVisualizacionesPorMarca', 
                 'checkout': 'cantidadDeCheckouts',
                 'conversion': 'cantidadConversiones',
                 'generic listing': 'cantidadDeVisualizacionesDeLaHome',
                 'lead': 'cantidadDeGeneracionDeNotificaciones',
                 'search engine hit': 'cantidadDeIngresosPorBuscador',
                 'searched products': 'cantidadDeProductosBuscados',
                 'staticpage': 'cantidadDeVisitas',
                 'viewed product': 'cantidadDeVisualizacionesDeProducto',
                 'visited site': 'cantidadDeIngresosPorUrl',
                })
scaler = MinMaxScaler()
for column in df:
    df[[column]] = scaler.fit_transform(df[[column]])

features = pd.merge(features, df, on='person', how= 'left')

In [23]:
#Features: Cantidad de visualizaciones por marca
eventos_con_marca = eventos.loc[eventos.event == 'viewed product']
eventos_con_marca['marca'] = eventos_con_marca['model'].str.split(' ').str[0]
df = eventos_con_marca.groupby('person')['marca'].value_counts().unstack()
df = df.fillna(0)
df.columns = ['cantidadDeVisualizacionesPor' + str(col) for col in df.columns]
scaler = MinMaxScaler()
for column in df:
    df[[column]] = scaler.fit_transform(df[[column]])
features = pd.merge(features, df, on='person', how= 'left')

C:\Users\felip\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [24]:
#Features: Realizo algun ingreso por tipo de canal proveniente generado
df = eventos.groupby('person')['channel'].value_counts().unstack()
df = df.fillna(0)
df.columns = ['realizoIngresoPorCanal' + str(col) for col in df.columns]

for column in df:
    df[column] = df[column].apply(lambda x: True if x>0 else False)
    
aux = pd.merge(labels, df, on='person', how= 'left')
aux = aux.fillna(False)
features = pd.merge(features, aux, on='person', how='left')



In [25]:
#Features: Realizo algun ingreso por tipo de campania proveniente  generado
df = eventos.groupby('person')['campaign_source'].value_counts().unstack()
df = df.fillna(0)
df.columns = ['realizoIngresoPorCamapania' + str(col) for col in df.columns]

for column in df:
    df[column] = df[column].apply(lambda x: True if x>0 else False)
    
aux = pd.merge(labels, df, on='person', how= 'left')
aux = aux.fillna(False)
features = pd.merge(features, aux, on='person', how='left')



In [26]:
#Features: Si alguna vez realizo una conversion

eventos_conversion = eventos.loc[eventos.event == 'conversion',]
df = eventos_conversion.groupby(['person'])['event'].agg('count').reset_index()
df['realizoConversion'] = df['event'].apply(lambda x: True if x>0 else False)
del df['event']
aux = pd.merge(labels, df, on='person', how= 'left')
#del aux['label']
aux = aux.fillna(False)
features = pd.merge(features, aux, on='person', how='left')


In [27]:
#Features: Si alguna vez realizo un checkout

eventos_conversion = eventos.loc[eventos.event == 'checkout',]
df = eventos_conversion.groupby(['person'])['event'].agg('count').reset_index()
df['realizoCheckout'] = df['event'].apply(lambda x: True if x>0 else False)
del df['event']
aux = pd.merge(labels, df, on='person', how= 'left')
aux = aux.fillna(False)
features = pd.merge(features, aux, on='person', how='left')


In [28]:
#Feature: Realizo algun ingreso por tipo de dispositivo
df = eventos.loc[eventos.event == 'visited site']
df = df.groupby(['person'])['device_type'].value_counts().unstack()

df.columns = ['realizoIngresoPorTipoDeDispositivo' + str(col) for col in df.columns]

for column in df:
    df[column] = df[column].apply(lambda x: True if x>0 else False)
    
aux = pd.merge(labels, df, on='person', how= 'left')
aux = aux.fillna(False)

features = pd.merge(features, aux, on='person', how='left')

In [29]:
#Feature: Es un usuario de brazil?
df = eventos.loc[eventos.event == 'visited site']
df = df.groupby(['person'])['country'].value_counts().unstack()
df['esUnUsuarioDeBrasil'] = df['Brazil'].apply(lambda x: True if x>0 else False)
df = df.loc[:,'esUnUsuarioDeBrasil']
df = df.reset_index()
aux = pd.merge(labels, df, on='person', how= 'left')
aux = aux.fillna(False)
features = pd.merge(features, aux, on='person', how='left')


In [16]:
#Feature: cantidad de busquedas de acuerdo al storage del dispositivo
df = eventos.loc[eventos.event == 'viewed product']
df = df.groupby(['person'])['storage'].value_counts().unstack()
df = df.fillna(0)
df.columns = ['cantidadDeVisualizacionesPorTipoAlmacenamientoDe' + str(col) for col in df.columns]
scaler = MinMaxScaler()
for column in df:
    df[[column]] = scaler.fit_transform(df[[column]])
features = pd.merge(features, df, on='person', how= 'left')

In [30]:
#Feature a que marca realizo la conversion. Dado que la mayoria son iphone, samsung  motorola vamos a armar una columna booleana con esas 3 marcas
eventos_con_marca = eventos
eventos_con_marca['marca'] = eventos['model'].str.split(' ').str[0]
df = eventos_con_marca.loc[eventos.event == 'conversion']
df = df.groupby(['person'])['marca'].value_counts().unstack()
df['realizoConversionAIphone'] = df['iPhone'].apply(lambda x: True if x>0 else False)
df['realizoConversionASamsung'] = df['Samsung'].apply(lambda x: True if x>0 else False)
df['realizoConversionAMotorola'] = df['Motorola'].apply(lambda x: True if x>0 else False)

df = df.loc[:,['realizoConversionAIphone','realizoConversionASamsung', 'realizoConversionAMotorola']]
df = df.reset_index()

aux = pd.merge(labels, df, on='person', how= 'left')

aux = aux.fillna(False)
features = pd.merge(features, aux, on='person', how='left')



In [31]:
#Feature a que marca realizo la checkout. Dado que la mayoria son iphone, samsung  motorola vamos a armar una columna booleana con esas 3 marcas
eventos_con_marca = eventos
eventos_con_marca['marca'] = eventos['model'].str.split(' ').str[0]
df = eventos_con_marca.loc[eventos.event == 'checkout']
df = df.groupby(['person'])['marca'].value_counts().unstack()
df['realizoCheckoutAIphone'] = df['iPhone'].apply(lambda x: True if x>0 else False)
df['realizoCheckoutASamsung'] = df['Samsung'].apply(lambda x: True if x>0 else False)
df['realizoCheckoutAMotorola'] = df['Motorola'].apply(lambda x: True if x>0 else False)
df['realizoCheckoutASony'] = df['Sony'].apply(lambda x: True if x>0 else False)

df = df.loc[:,['realizoCheckoutAIphone','realizoCheckoutASamsung', 'realizoCheckoutAMotorola','realizoCheckoutASony']]
df = df.reset_index()

aux = pd.merge(labels, df, on='person', how= 'left')
aux = aux.fillna(False)
features = pd.merge(features, aux, on='person', how='left')


In [32]:
#Compleamos los campos NA con 0
features = features.fillna(0)

In [33]:
list(features)

['person',
 'retornoAlSitio',
 'cantidadVisualizacionesBuenEstado',
 'cantidadVisualizacionesBuenEstadoSinTouch',
 'cantidadVisualizacionesExcelente',
 'cantidadVisualizacionesMuyBuenEstado',
 'cantidadVisualizacionNuevo',
 'cantidad_eventos_generados',
 'cantidadIngresosPorCampania',
 'cantidadDeVisualizacionesPorMarca',
 'cantidadDeCheckouts',
 'cantidadConversiones',
 'cantidadDeVisualizacionesDeLaHome',
 'cantidadDeGeneracionDeNotificaciones',
 'cantidadDeIngresosPorBuscador',
 'cantidadDeProductosBuscados',
 'cantidadDeVisitas',
 'cantidadDeVisualizacionesDeProducto',
 'cantidadDeIngresosPorUrl',
 'cantidadDeVisualizacionesPorAsus',
 'cantidadDeVisualizacionesPorLG',
 'cantidadDeVisualizacionesPorLenovo',
 'cantidadDeVisualizacionesPorMotorola',
 'cantidadDeVisualizacionesPorQuantum',
 'cantidadDeVisualizacionesPorSamsung',
 'cantidadDeVisualizacionesPorSony',
 'cantidadDeVisualizacionesPoriPad',
 'cantidadDeVisualizacionesPoriPhone',
 'realizoIngresoPorCanalDirect',
 'realizoIngr

In [34]:
features.to_csv("../set_a_predecir-definitivo.csv")

In [35]:
features.sample(30)

,person,retornoAlSitio,cantidadVisualizacionesBuenEstado,cantidadVisualizacionesBuenEstadoSinTouch,cantidadVisualizacionesExcelente,cantidadVisualizacionesMuyBuenEstado,cantidadVisualizacionNuevo,cantidad_eventos_generados,cantidadIngresosPorCampania,cantidadDeVisualizacionesPorMarca,...,realizoIngresoPorTipoDeDispositivoTablet,realizoIngresoPorTipoDeDispositivoUnknown,esUnUsuarioDeBrasil,realizoConversionAIphone,realizoConversionASamsung,realizoConversionAMotorola,realizoCheckoutAIphone,realizoCheckoutASamsung,realizoCheckoutAMotorola,realizoCheckoutASony
17885,9549b7c7,True,0.000000,0.000000,0.000000,0.000000,0.000000,0.000225,0.000000,0.000000,...,False,False,True,False,False,False,False,True,False,False
18806,e1ea2900,True,0.000000,0.000000,0.000000,0.001910,0.000000,0.000902,0.001912,0.000000,...,False,False,True,False,False,False,True,False,False,False
13704,2fc44479,True,0.025202,0.080645,0.018957,0.009551,0.000000,0.024115,0.030593,0.005817,...,False,False,True,False,False,False,False,True,False,False
17473,64fe2656,True,0.002016,0.000000,0.000000,0.001910,0.000000,0.002028,0.001912,0.000000,...,False,False,True,False,False,False,True,False,False,False
16956,91522189,True,0.005040,0.016129,0.002844,0.002865,0.000000,0.004733,0.001912,0.002115,...,False,False,True,False,False,False,False,False,False,False
7638,1067e372,True,0.002016,0.000000,0.000000,0.001910,0.000000,0.004733,0.000000,0.000000,...,False,False,True,False,True,False,False,True,False,False
9257,2705b203,True,0.001008,0.000000,0.000000,0.000000,0.000000,0.000676,0.001912,0.000000,...,False,False,True,False,False,False,False,True,False,False
7025,8e6de18a,True,0.001008,0.000000,0.007583,0.004776,0.000000,0.004958,0.001912,0.000529,...,False,False,True,False,False,False,True,False,False,False
7690,ab8da6c6,True,0.002016,0.000000,0.000948,0.000000,0.000000,0.001127,0.003824,0.000000,...,False,False,True,False,False,False,False,False,False,False
14000,8dfb43d6,True,0.014113,0.008065,0.001896,0.007641,0.000000,0.010367,0.000000,0.000529,...,False,False,True,False,False,False,False,False,False,False
